# creating interactive maps with folium and geopandas

- create the map object using geopandas explore
    - load wards data
    - use join (not spatial join) to add additional data?
         - number of parks?
         - park area?
         - calculate park area per capita?
         - something to do with public transit?

- add some additional data to the map
    - bus/train stations? have to give them the link to the data, then download it.
    - make the tooltip fancy (Name: , Link: , etc)

- changing some different features
    - e.g., tooltips/popups, etc.
    - changing colormaps (link to matplotlib colormaps)

- export to html
    - show an example page hosted on github

- additional exercise: make a map that ...


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium


def create_marker(row, folium_map):
    """
    Create an airport marker from a row of a pandas DataFrame.    
    """
    marker = folium.Marker([row['lat'], row['lon']],
                           icon=folium.Icon(color='red', icon='plane', prefix='fa'))   
    marker.add_to(folium_map)


load the data using geopandas:

In [ ]:
wards = gpd.read_file('data_files/NI_Wards.shp')

see what we're working with using head()

In [ ]:
wards.head()

now, create a folium map object using geopandas.GeoDataFrame.explore:

In [ ]:
m = wards.explore('Population', cmap='viridis')
m # to show the map

add some point data to the map - maybe points for airports?

In [ ]:
airports = pd.read_csv('data_files/Airports.csv')
airports['geometry'] = list(zip(airports['lon'], airports['lat']))
airports['geometry'] = airports['geometry'].apply(Point)

airports = gpd.GeoDataFrame(airports)
airports.set_crs(epsg=4326, inplace=True)

In [ ]:
airports[['name', 'website', 'geometry']].explore('name', m=m, marker_type='marker', popup=True, legend=False,
                                                  marker_kwds={'icon': folium.Icon(color='red', icon='plane', prefix='fa')})

load some other data using pandas. Note that this doesn't have any geospatial information, but it does have a ward code.

In [ ]:
transport = pd.read_csv('data_files/transport_data.csv')
transport.head()

join the transport data to the wards data using the ward code and `geopandas.GeoDataFrame.merge()`

In [ ]:
merged = wards.merge(transport, left_on='Ward Code', right_on='Ward Code')
merged.head()

now, re-create the map using the transport data joined to the wards, and add the airport markers like we used before

In [ ]:
m = merged.explore('Distance', cmap='plasma', 
                   legend_kwds={'caption': 'Distance to nearest bus/rail station in km'})

airports[['name', 'website', 'geometry']].explore('name', m=m, marker_type='marker', popup=True, legend=False,
                                                  marker_kwds={'icon': folium.Icon(color='red', icon='plane', prefix='fa')})

m # show the map

save the final map to an html file:

In [ ]:
m.save('test.html')

Additional examples:

- add a dropdown to select the base layer (does it save to the html)?
- maybe an "antpath" example?
- marker styles?